In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

import numpy as np

In [ ]:
from network.model import NerFormer
from network.feature_network import FeatureNet

In [ ]:
from typing import Union, IO, Any, List, Tuple, Optional
import typing
import json
import gzip

from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
frame_annotations_file = "test_dataset/frame_annotations.jgz"
frame_zipfile = gzip.open(frame_annotations_file, "rt", encoding="utf8")

frame_dicts = json.load(frame_zipfile)
print(type(frame_dicts), len(frame_dicts))

sequence_annotations_file = "test_dataset/test_sequence_annotations.jgz"
seq_zipfile = gzip.open(sequence_annotations_file, "rt", encoding="utf8")

seq_dicts = json.load(seq_zipfile)
print(type(seq_dicts), len(seq_dicts))

In [ ]:
print(seq_dicts[13])

In [ ]:
# test dataset image : 1323 ~ 1332
data_dicts = frame_dicts[1323:1333]
print(data_dicts[0], '\n\n', data_dicts[-1])

print('\n\n', data_dicts[0].keys())
print('\n', data_dicts[0]['viewpoint'])

In [ ]:
img1 = Image.open("test_dataset/frame000001.jpg")

print(img1.size)
plt.imshow(img1)

In [ ]:
intrinsic = torch.eye(4)

viewpoint = data_dicts[0]['viewpoint']
intrinsic[0][0], intrinsic[1][1] = viewpoint['focal_length'][0], viewpoint['focal_length'][1]
intrinsic[0][2], intrinsic[1][2] = viewpoint['principal_point'][0], viewpoint['principal_point'][1]
intrinsic[2, 2] = 1

print(intrinsic)

In [ ]:
extrinsic = torch.eye(4)

rotation = torch.tensor(viewpoint['R'])
translation = torch.tensor(viewpoint['T'])

print(rotation)
print(translation)
print(rotation.shape, translation.shape)

extrinsic[:3, :3] = rotation
extrinsic[:3, 3] = translation

print(extrinsic)
print(extrinsic.shape)

In [ ]:
c2w = torch.inverse(intrinsic.mul(extrinsic))

print(c2w)
print(c2w.shape)

In [ ]:
class HarmonicEmbedding(torch.nn.Module):
    def __init__(self, n_harmonic_functions=60, omega0=0.1):
        """
        Given an input tensor `x` of shape [minibatch, ... , dim],
        the harmonic embedding layer converts each feature
        in `x` into a series of harmonic features `embedding`
        as follows:
            embedding[..., i*dim:(i+1)*dim] = [
                sin(x[..., i]),
                sin(2*x[..., i]),
                sin(4*x[..., i]),
                ...
                sin(2**(self.n_harmonic_functions-1) * x[..., i]),
                cos(x[..., i]),
                cos(2*x[..., i]),
                cos(4*x[..., i]),
                ...
                cos(2**(self.n_harmonic_functions-1) * x[..., i])
            ]
            
        Note that `x` is also premultiplied by `omega0` before
        evaluating the harmonic functions.
        """
        super().__init__()
        self.register_buffer(
            'frequencies',
            omega0 * (2.0 ** torch.arange(n_harmonic_functions)),
        )
        
    def forward(self, x):
        """
        Args:
            x: tensor of shape [..., dim]
        Returns:
            embedding: a harmonic embedding of `x`
                of shape [..., n_harmonic_functions * dim * 2]
        """
        embed = (x[..., None] * self.frequencies).view(*x.shape[:-1], -1)

        return torch.cat((embed.sin(), embed.cos()), dim=-1)

In [ ]:
pe = HarmonicEmbedding(10)

pe_test = pe(torch.tensor([0, 0, 1]))
print(pe_test)
print(pe_test.shape)


In [ ]:
# 타겟 이미지 내에서 랜덤으로 픽셀 샘플링
def sample_random_pixel(H, W, N_rand, sample_mode, center_ratio=0.8):
    rng = np.random.RandomState(234)

    if sample_mode == 'center':
        border_H = int(H * (1 - center_ratio) / 2.)
        border_W = int(W * (1 - center_ratio) / 2.)

        # pixel coordinates
        u, v = np.meshgrid(np.arange(border_H, H - border_H),
                            np.arange(border_W, W - border_W))
        u = u.reshape(-1)
        v = v.reshape(-1)

        select_inds = rng.choice(u.shape[0], size=(N_rand,), replace=False)
        select_inds = v[select_inds] + W * u[select_inds]

    elif sample_mode == 'uniform':
        # Random from one image
        select_inds = rng.choice(H*W, size=(N_rand,), replace=False)
    else:
        raise Exception("unknown sample mode!")

    return select_inds

In [ ]:
random_pixels = sample_random_pixel(500, 500, 800, "center")
print(len(random_pixels), '\n', random_pixels[:10])

In [ ]:
# 타겟 이미지의 각 픽셀마다 rays_o, rays_d를 구함
def get_rays(H, W, intrinsics, c2w, batch_size=1, render_stride=1):
    '''
    param H: image height
    param W: image width
    param intrinsics: 4 by 4 intrinsic matrix
    param c2w: 4 by 4 camera to world extrinsic matrix
    
    return: rays_o, rays_d
    '''
    # u --> x 인덱싱
    # v --> y 인덱싱
    u, v = np.meshgrid(np.arange(W)[::render_stride], np.arange(H)[::render_stride])

    # 이미지의 각 row들이 이어붙은 형태로 변환
    # (H, W) --> (H*W)
    u = u.reshape(-1).astype(dtype=np.float32)  # + 0.5    # add half pixel
    v = v.reshape(-1).astype(dtype=np.float32)  # + 0.5

    pixels = np.stack((u, v, np.ones_like(u)), axis=0)  # [3(x+y+z), H*W]
    pixels = torch.from_numpy(pixels)
    print(pixels.shape)
    batched_pixels = pixels.unsqueeze(0).repeat(batch_size, 1, 1)

    # bmm : batch matrix-matrix product 
    # [B, N, M] tensor * [B, M, P] tensor >>> [B, N, P]
    rays_d = (c2w[:, :3, :3].bmm(torch.inverse(intrinsics[:, :3, :3])).bmm(batched_pixels)).transpose(1, 2)
    # rays_d = (c2w[:, :3, :3].bmm(torch.inverse(intrinsics[:, :3, :3])).bmm(pixels)).transpose(1, 2)
    rays_d = rays_d.reshape(-1, 3)
    rays_o = c2w[:, :3, 3].unsqueeze(1).repeat(1, rays_d.shape[0], 1).reshape(-1, 3)  # B x HW x 3
    
    return rays_o, rays_d

In [ ]:
rays_o, rays_d = get_rays(img1.height, img1.width, intrinsic.unsqueeze(0), c2w.unsqueeze(0))

print(rays_o.shape, rays_d.shape)

In [150]:
def sample_along_camera_ray(ray_o, ray_d, depth_range,
                            N_samples,
                            inv_uniform=False,
                            det=False):
    '''
    param ray_o: scene coordinate system에서 ray의 원점 : [N_rays, 3]
    param ray_d: scene coordinate system에서 ray의 direction vector : [N_rays, 3]
    param depth_range: [near_depth, far_depth]
    param inv_uniform: if True, uniformly sampling inverse depth
    param det: if True, will perform deterministic sampling
    
    return: tensor of shape [N_rays, N_samples, 3]
    '''
    # 샘플들은 [near_depth, far_depth] 범위 안에 있어야 함.
    # assume the nearest possible depth is at least (min_ratio * depth)
    near_depth_value = depth_range[0, 0]
    far_depth_value = depth_range[0, 1]
    assert near_depth_value > 0 and far_depth_value > 0 and far_depth_value > near_depth_value

    near_depth = near_depth_value * torch.ones_like(ray_d[..., 0])
    far_depth = far_depth_value * torch.ones_like(ray_d[..., 0])

    if inv_uniform:
        start = 1. / near_depth     # [N_rays,]
        step = (1. / far_depth - start) / (N_samples-1)
        inv_z_vals = torch.stack([start+i*step for i in range(N_samples)], dim=1)  # [N_rays, N_samples]
        z_vals = 1. / inv_z_vals
    else:
        start = near_depth
        step = (far_depth - near_depth) / (N_samples-1)
        z_vals = torch.stack([start+i*step for i in range(N_samples)], dim=1)  # [N_rays, N_samples]

    if not det:
        # get intervals between samples
        mids = .5 * (z_vals[:, 1:] + z_vals[:, :-1])
        upper = torch.cat([mids, z_vals[:, -1:]], dim=-1)
        lower = torch.cat([z_vals[:, 0:1], mids], dim=-1)
        # uniform samples in those intervals
        t_rand = torch.rand_like(z_vals)
        z_vals = lower + (upper - lower) * t_rand   # [N_rays, N_samples]

    ray_d = ray_d.unsqueeze(1).repeat(1, N_samples, 1)  # [N_rays, N_samples, 3]
    ray_o = ray_o.unsqueeze(1).repeat(1, N_samples, 1)

    # 샘플 포인트 위치를 계산.
    # ray의 원점으로부터 ray의 방향으로 z_val만큼 전진한 것.
    pts = z_vals.unsqueeze(2) * ray_d + ray_o       # [N_rays, N_samples, 3]
    
    return pts, z_vals

In [145]:
ray_d = rays_d.clone()
ray_o = rays_o.clone()

print(ray_d.shape, ray_o.shape)

torch.Size([171961, 3]) torch.Size([171961, 3])


In [154]:
depth_range = torch.tensor([[0.1, 3.0]])
N_samples = 20

pts, z_vals = sample_along_camera_ray(ray_o, ray_d, depth_range, N_samples)

In [155]:
print(pts.shape)
print(z_vals.shape)

torch.Size([171961, 20, 3])
torch.Size([171961, 20])


In [159]:
# 400번째 ray에서의 samples
pts[400]

tensor([[ -1.4153,  -0.0613,   0.1650],
        [ -1.5882,  -0.0688,   0.1851],
        [ -3.4004,  -0.1472,   0.3963],
        [ -5.2240,  -0.2261,   0.6089],
        [ -6.6318,  -0.2871,   0.7730],
        [ -7.8613,  -0.3403,   0.9163],
        [ -8.7938,  -0.3807,   1.0250],
        [-10.1739,  -0.4404,   1.1858],
        [-10.7546,  -0.4656,   1.2535],
        [-12.8424,  -0.5560,   1.4969],
        [-13.9064,  -0.6020,   1.6209],
        [-15.5691,  -0.6740,   1.8147],
        [-16.1829,  -0.7006,   1.8862],
        [-17.7843,  -0.7699,   2.0729],
        [-18.9395,  -0.8199,   2.2075],
        [-20.2334,  -0.8759,   2.3583],
        [-21.3119,  -0.9226,   2.4840],
        [-22.5580,  -0.9765,   2.6293],
        [-24.7361,  -1.0708,   2.8831],
        [-25.7066,  -1.1128,   2.9963]])